### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow
import keras
import nltk
import gensim

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout,SpatialDropout1D, Bidirectional,Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split

from scipy.cluster.vq import whiten

from collections import Counter

from gensim.models import Word2Vec
from gensim.models import FastText

Using TensorFlow backend.


In [0]:
import string
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split
import io
import matplotlib.pyplot as plt

from __future__ import print_function
from __future__ import division
import json
import numpy as np

from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Dropout, Flatten, Lambda, Embedding, LSTM,Bidirectional, concatenate, BatchNormalization 
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.initializers import RandomNormal
import tensorflow as tf
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [0]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

### Code

In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
from pathlib import Path
import pandas as pd

path = Path(os.getcwd())

df = pd.read_csv(path/'gdrive'/'My Drive'/'ML'/'workshop materials'/'bangla_lstm.csv');

In [5]:
df.head()

,post,tags
0,ভুঁইফোড় দুটি সংগঠনকে আইএসআইয়ের অর্থায়ন! বাংলাদ...,mk
1,আজকের দিনটা এত সুন্দর কেন !! সকাল নয়টা। গুল্লু...,rn
2,ডিসেম্বর _ ১৯৭১ থেকে ২০১৬ ডিসেম্বর : ১৯৭১ থেকে...,fe
3,পিচ্চিতোষ গল্প ১৩: অনেক রকম গন্ধ ঘুম থেকে উঠে ...,hm
4,ইংলিশ ক্যাপ্টেন বাটলারের বাটপারি!!! লড়াই এবার ...,rg


In [0]:
max_length = 50
vocab_size=50

lst = df['post'].to_list()
encoded_docs = [one_hot(d, vocab_size) for d in lst]
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [32]:
len(encoded_docs)

2100

In [33]:
encoded_docs[0][:10]

[2, 17, 5, 10, 41, 27, 17, 5, 21, 43]

In [34]:
len(encoded_docs[0]), len(encoded_docs[1]) , len(encoded_docs[2])

(468, 894, 1120)

One-hot encode a text into a list of word indexes in a vocabulary of size n.



In [35]:
len(padded_docs[0]), len(padded_docs[1]) , len(padded_docs[2])

(50, 50, 50)

In [0]:
xtrain, xtest, ytrain, ytest = train_test_split(padded_docs, df['tags'] , test_size=0.4 , random_state = 52)

In [0]:
ytrain

In [0]:
ytest

In [0]:
xtrain

In [0]:
xtest

In [23]:
xtrain.shape, xtest.shape

((1260, 50), (840, 50))

In [24]:
df.tags.unique()

array(['mk', 'rn', 'fe', 'hm', 'rg', 'ij'], dtype=object)

In [0]:
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(ytrain)

In [0]:
y_test=encoder.transform(ytest) ## notice no fit

In [28]:
y_train

array([0, 1, 2, ..., 4, 5, 0])

In [29]:
y_train.shape

(1260,)

In [30]:
y_train=y_train.reshape(-1, 1)
y_test=y_test.reshape(-1, 1)

y_train.shape, y_test.shape

((1260, 1), (840, 1))

In [31]:
onehotencoder = preprocessing.OneHotEncoder()

y_train = onehotencoder.fit_transform(y_train).toarray()
y_test = onehotencoder.transform(y_test).toarray()

y_train.shape , y_test.shape

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


((1260, 6), (840, 6))

In [0]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [0]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [40]:
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           5000      
_________________________________________________________________
flatten_2 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)                100020    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 66        
Total params: 105,296
Trainable params: 105,296
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
# fit the model
model.fit(xtrain, y_train, epochs=5)

Epoch 1/5
1260/1260 [==============================] - 0s 285us/step - loss: 0.0368 - acc: 1.0000
Epoch 2/5
1260/1260 [==============================] - 0s 110us/step - loss: 0.0132 - acc: 1.0000
Epoch 3/5
1260/1260 [==============================] - 0s 112us/step - loss: 0.0064 - acc: 1.0000
Epoch 4/5
1260/1260 [==============================] - 0s 110us/step - loss: 0.0039 - acc: 1.0000
Epoch 5/5
1260/1260 [==============================] - 0s 108us/step - loss: 0.0026 - acc: 1.0000


In [46]:
# evaluate the model
loss, accuracy = model.evaluate(xtrain, y_train)
print('Accuracy: %f' % (accuracy*100))

1260/1260 [==============================] - 0s 85us/step
Accuracy: 100.000000


In [47]:
# evaluate the model for tes data
loss, accuracy = model.evaluate(xtest, y_test)
print('Accuracy: %f' % (accuracy*100))

840/840 [==============================] - 0s 46us/step
Accuracy: 50.119048
